In [0]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install tensorflow
!pip install librosa
!pip install keras

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras

#from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Activation, Dense,  Embedding

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout#, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

from keras import regularizers
import os
import pandas as pd
from google.colab import drive


In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def extract_feature(file_name, offst=0.1):
	X, sample_rate = librosa.load(
		file_name, res_type='kaiser_fast', offset=offst)
	mfccs = np.mean(librosa.feature.mfcc(
		y=X, sr=sample_rate, n_mfcc=64).T, axis=0)
	return mfccs

In [0]:
def getPrediction(eng_stress_model, livecnn):
    #predict!
    livepreds = eng_stress_model.predict(livecnn, 
                             batch_size=32, 
                             verbose=1)

    #convert prediction to text
    livepreds1 = livepreds.argmax(axis=1)
    liveabc = livepreds1.astype(int).flatten()
    conv = ['female_not_stressed', 'female_stressed', 'male_not_stressed',
           'male_stressed']

    #array for conversion into binary class (stressed/not_stressed)
    conv2 = ['not_stressed', 'stressed', 'not_stressed', 'stressed']

    #prediction with 4 classes
    livepredictions4 = conv[liveabc[0]]
    #prediction with 2 classes
    livepredictions2 = conv2[liveabc[0]]
    
    return livepredictions2

In [0]:
mfccs = extract_feature(
	'drive/My Drive/Colab Notebooks/saved_models/Examples/Angry_female.wav', 0)

live = pd.DataFrame(data=mfccs)
live = live.stack().to_frame().T

livecnn = np.expand_dims(live, axis=2)

In [0]:
path_to_users_model_folder = 'drive/My Drive/Colab Notebooks/saved_models/model#userID'
path_to_def_model_folder = 'drive/My Drive/Colab Notebooks/saved_models/MFCC64_ENG'

In [0]:
if os.path.isdir(path_to_users_model_folder):
    print("exists")
    model_path = path_to_users_model_folder + '/' + 'userID_Stress_recog_64MFCC_with_gender.json'
    weights_path = path_to_users_model_folder + '/' + 'userID_Stress_recog_64MFCC_with_gender.h5'
else:
    print("not exists")
    model_path = path_to_def_model_folder + '/' + 'Stress_recog_64MFCC_with_gender.json'
    weights_path = path_to_def_model_folder + '/' + 'Stress_recog_64MFCC_with_gender.h5'

not exists


In [0]:
# loading json and creating model with 4 classes
from keras.models import model_from_json
json_file5 = open(model_path, 'r')
eng2 = json_file5.read()
json_file5.close()
eng_stress_model = model_from_json(eng2)

# load weights into new model
eng_stress_model.load_weights(weights_path)
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [0]:
#compile the model
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
eng_stress_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

livepredictions2 = getPrediction(eng_stress_model, livecnn)

print(livepredictions2)

1/1 [==============================] - 0s 216ms/step
stressed


In [0]:
# feedback and user_male are info that is taken from the user!!!!!!!!!!!!!!!!
feedback =  #'stressed'
user_male =  #True
epNum = 0
while(True):
  if feedback != livepredictions2:
      if user_male == True:
          it = (liveabc[0] + 1) % 2 + 2
      else:
          it = (liveabc[0] + 1) % 2
      
      for layer in eng_stress_model.layers[:16]:
          layer.trainable=False
      for layer in eng_stress_model.layers[16:]:
          layer.trainable=True
      
      eng_stress_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

      y_train = np.zeros((1, 4))
      y_train[0][it] = 1
      
      eng_stress_model.fit(livecnn, y_train, batch_size=32, epochs=1, validation_data=(livecnn, y_train))
      
      livepredictions2 = getPrediction(eng_stress_model, livecnn)
      epNum += 1

  else:
      break
  print(epNum)
if epNum > 0:
  user_model_path = path_to_users_model_folder + '/' + 'userID_Stress_recog_64MFCC_with_gender.json'
  user_weights_path = path_to_users_model_folder + '/' + 'userID_Stress_recog_64MFCC_with_gender.h5'
  
  model_name = 'userID_Stress_recog_64MFCC_with_gender.h5'
  save_dir = path_to_users_model_folder
  # Save model and weights
  if not os.path.isdir(save_dir):
      os.makedirs(save_dir)
  model_path = os.path.join(save_dir, model_name)
  eng_stress_model.save(model_path)
  print('Saved trained model at %s ' % model_path)
  
  import json
  model_json = eng_stress_model.to_json()
  with open(user_model_path, "w") as json_file:
      json_file.write(model_json)



VSE
Saved trained model at drive/My Drive/Colab Notebooks/saved_models/model#userID/userID_Stress_recog_64MFCC_with_gender.h5 


In [0]:
model_name = 'userID_Stress_recog_64MFCC_with_gender.h5'
save_dir = path_to_users_model_folder
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
eng_stress_model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at drive/My Drive/Colab Notebooks/saved_models/Stress_recog_MFCC_with_gender.h5 


In [0]:
import json
model_json = model.to_json()
with open("drive/My Drive/Colab Notebooks/saved_models/Stress_recog_MFCC_with_gender.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
for i,layer in enumerate(eng_stress_model.layers):
  print(i,layer.name)

In [0]:
for layer in eng_stress_model.layers[:16]:
    layer.trainable=False
for layer in eng_stress_model.layers[16:]:
    layer.trainable=True

In [0]:
for i,layer in enumerate(eng_stress_model.layers):
    print(i,layer.trainable)

In [0]:
eng_stress_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

y_train = np.zeros((1, 4))
y_train[it] = 1


eng_stress_model.fit(livecnn, y_train, batch_size=32, epochs=1, validation_data=(livecnn, y_train))

In [0]:
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', offset=offst)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

  def extract_feature_lite(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr=None)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=30).T, axis=0)
    
    return mfccs

In [0]:

mfccs = #array of input features. dimensions: 1x64!!!!

live = pd.DataFrame(data=mfccs)
live = live.stack().to_frame().T



In [0]:
livecnn = np.expand_dims(live, axis=2)

In [0]:
# loading json and creating model with 4 classes
from keras.models import model_from_json

json_file5 = open('drive/My Drive/Colab Notebooks/saved_models/Stress_recog_MFCC_with_gender.json', 'r')

eng2 = json_file5.read()
json_file5.close()

eng_stress_model = model_from_json(eng2)

# load weights into new model
eng_stress_model.load_weights("drive/My Drive/Colab Notebooks/saved_models/Stress_recog_MFCC_with_gender.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
eng_stress_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
livepreds = eng_stress_model.predict(livecnn, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 0s 138ms/step


In [0]:
livepreds1 = livepreds.argmax(axis=1)
liveabc = livepreds1.astype(int).flatten()
conv = ['female_not_stressed', 'female_stressed', 'male_not_stressed',
       'male_stressed']

#array for conversion into binary class (stressed/not_stressed)
conv2 = ['not_stressed', 'stressed', 'not_stressed', 'stressed']

livepredictions = conv[liveabc[0]]
print(livepredictions)

female_stressed


In [0]:
import pandas as pd
df_reader = pd.read_table("drive/My Drive/Colab Notebooks/Features/ENG8_STRESS_MFCC_GENDER.csv", sep='\t', encoding='utf-8')
file_df = pd.DataFrame(df_reader)
file_df = file_df.drop('Unnamed: 0', axis = 1)

In [0]:
newdf1 = np.random.rand(len(file_df)) > 0.2

train = file_df[newdf1]
test = file_df[~newdf1]

print(len(train), len(test))

1161 279


In [0]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]

testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [0]:
print(trainlabel.shape, testlabel.shape)

(1161, 1) (279, 1)


In [0]:
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
eng_stress_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = eng_stress_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (eng_stress_model.metrics_names[1], score[1]*100))

acc: 97.85%


In [0]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)

X_test = np.array(testfeatures)
y_test = np.array(testlabel)


print(X_train.shape)
print(X_test.shape)

y_train = np.squeeze(y_train, axis=(1,))
y_test = np.squeeze(y_test, axis=(1,))

(1161, 64)
(279, 64)


In [0]:
trainfeatures.columns.values

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63'], dtype=object)

In [0]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))

In [0]:
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [0]:
print(y_train.shape)
print(y_test.shape)
print(y_train)

(1161, 4)
(279, 4)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [0]:
np.save('drive/My Drive/Colab Notebooks/saved_models/4class_stress_with_gender_classes.npy', lb.classes_)

In [0]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)
print(x_testcnn.shape)

(279, 64, 1)


In [0]:
lb.classes_

array(['female_not_stressed', 'female_stressed', 'male_not_stressed',
       'male_stressed'], dtype=object)